In [ ]:
import numpy as np

grid_size = (5,5,5)
n_points = np.prod(grid_size)
params = np.zeros(np.prod(grid_size)*3)
for i, p in enumerate(np.ndindex(grid_size)):
    params[i] = p[2]
    params[n_points + i] = p[1]
    params[2*n_points + i] = p[0]

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

x = np.loadtxt("points.txt")
spacing = [3, 3, 3]
size = [13, 13, 13]
slice = 6

plt.figure(figsize=(8,8))
for p in x:
    if p[2] == slice:
        plt.scatter(p[3], p[4], c="r")

In [3]:
%%timeit -r 15

from pathlib import Path
from thesispy.elastix_wrapper.wrapper import run
from thesispy.elastix_wrapper.parameters import Parameters
from thesispy.definitions import Collection, ROOT_DIR
from thesispy.experiments.validation import plot_cpoints

params_main = (
        Parameters.from_base(mesh_size=4, metric="AdvancedMeanSquares", seed=2)
        .asgd()
        .regularize(0.01)
        .stopping_criteria(200)
        .instance(Collection.SYNTHETIC, 1)
)
_ = run(params_main, ROOT_DIR / "output/" / str(params_main), suppress_stdout=True, visualize=False, validate=False)

2022-10-25 16:17:48,236 - Wrapper - INFO - Running elastix in: /home/joasiee/Documents/projects/elastix/experiments/output/1666707468_synthetic_1_adaptivestochasticgradientdescent_235948
2022-10-25 16:17:48,949 - Wrapper - INFO - Run ended. It took 0.7131 seconds
2022-10-25 16:17:48,950 - Wrapper - INFO - Running elastix in: /home/joasiee/Documents/projects/elastix/experiments/output/1666707468_synthetic_1_adaptivestochasticgradientdescent_949969
2022-10-25 16:17:49,883 - Wrapper - INFO - Run ended. It took 0.9337 seconds
2022-10-25 16:17:49,885 - Wrapper - INFO - Running elastix in: /home/joasiee/Documents/projects/elastix/experiments/output/1666707469_synthetic_1_adaptivestochasticgradientdescent_884638
2022-10-25 16:17:50,705 - Wrapper - INFO - Run ended. It took 0.8201 seconds
2022-10-25 16:17:50,706 - Wrapper - INFO - Running elastix in: /home/joasiee/Documents/projects/elastix/experiments/output/1666707470_synthetic_1_adaptivestochasticgradientdescent_705715
2022-10-25 16:17:51,5

811 ms ± 47.4 ms per loop (mean ± std. dev. of 15 runs, 1 loop each)
